In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import joblib, json, sys, os
from IPython.display import Image, display_png

sys.path.append("../script/")
from MergeBrain import WholeBrainCells
import matplotlib.pyplot as plt
import pandas as pd

import sys, os, os.path, re, csv, math
import linecache
import numpy as  np
import pandas as pd
from matplotlib import pyplot as plt
from statistics import mean,stdev
import seaborn as sns

import csv
import pprint
import ants
import tifffile
from multiprocessing import Pool
import datetime
from itertools import product

import numpy as np
import tifffile
import os
import pickle
import joblib
import pandas as pd
from matplotlib import pyplot as plt
import concurrent.futures
import gc
import json

In [2]:

dt_local = np.dtype([
    ('local_x', 'f4'), ('local_y', 'f4'), ('local_z', 'f4'),
    ('structureness', 'f4'), ('blobness', 'f4'), ('intensity', 'f4'),
    ('size', 'u2'), ('padding', 'u2'), ('intensity2', 'f4'), ('intensity3', 'f4')
])


dt_annotated = np.dtype([
    ('stitched_x', 'f4'), ('stitched_y', 'f4'), ('stitched_z', 'f4'),
    ('mapped_x', 'f4'), ('mapped_y', 'f4'), ('mapped_z', 'f4'),
    ('atlas_id', 'u2'), ('is_positive','f4'),('is_positive2','f4'),('is_positive3','f4')
])



dt_annotated_init = np.dtype([
    ('local_x', 'f4'), ('local_y', 'f4'), ('local_z', 'f4'),
    ('stitched_x', 'f4'), ('stitched_y', 'f4'), ('stitched_z', 'f4'),
    ('mapped_x', 'f4'), ('mapped_y', 'f4'), ('mapped_z', 'f4'),
    ('atlas_id', 'u2'), ('is_positive','f4'),('intensity','f4'),('structureness','f4'), # ここのis_positiveはinitial classifyによるもの
    ('intensity_2','f4'), ('intensity_3','f4')
])

dt_annotated_classified_intensity = np.dtype([
    ('local_x', 'f4'), ('local_y', 'f4'), ('local_z', 'f4'),
    ('stitched_x', 'f4'), ('stitched_y', 'f4'), ('stitched_z', 'f4'),
    ('mapped_x', 'f4'), ('mapped_y', 'f4'), ('mapped_z', 'f4'),
    ('atlas_id', 'u2'), ('is_positive','f4'), ('intensity','f4'),('structureness','f4'),
    ('intensity2','f4'),('intensity3','f4'), # ここのis_positiveはinitial classifyによるものにする
    ('norm_intensity_2','f4'), ('norm_intensity_3','f4')
])

# 必要なdtypeの定義
dt_needed = np.dtype([
    ('mapped_x', 'f4'), ('mapped_y', 'f4'), ('mapped_z', 'f4'),
    ('intensity','f4'),('structureness','f4')
])

dt_classified = np.dtype([
    ('is_positive','bool'), ('is_positive2','bool'),('is_positive3','bool')
])

dt_classified2 = np.dtype([
   ('is_positive','bool'), ('intensity','f4'),('intensity_2','f4'), ('intensity_3','f4')
])

dt_annotation2 = np.dtype([
  ('stitched_x', 'f4'), ('stitched_y', 'f4'), ('stitched_z', 'f4'),
  ('mapped_x', 'f4'), ('mapped_y', 'f4'), ('mapped_z', 'f4'),
  ('SCA_x', 'f4'), ('SCA_y', 'f4'), ('SCA_z', 'f4'),
  ('atlas_id', 'u2')
])

def process_file_all (fw_dir_nu_, file_name):
    
    try:
        
        
        fw_dir_intensities_ = fw_dir_nu_.replace("nu_R_/result", "intensities_")
        tmp_points_intensities_ = np.fromfile(os.path.join(fw_dir_intensities_, file_name), dtype=dt_annotated_classified_intensity)
        
        
        #print(len( tmp_points_intensities_))
        tmp_src = np.zeros((len(tmp_points_intensities_),), dtype=dt_annotation2)
        
        fw_dir_nu_R_ = fw_dir_nu_ #.replace("intensities_","nu_R_/result")
        tmp_points_nu_R_ = np.fromfile(os.path.join(fw_dir_nu_R_, file_name), dtype=dt_annotated_init)
        
        mask = (tmp_points_nu_R_["stitched_x"] != 0.0) & (tmp_points_nu_R_["stitched_y"] != 0.0) & (tmp_points_nu_R_["is_positive"] > 0.0)
        #mask =  data_local_nucl["atlas_id"] > 0.0
        tmp_points_nu_R_2 = tmp_points_nu_R_[mask]
        

        tmp_src ["stitched_x"] = tmp_points_nu_R_2 ["stitched_x"]
        tmp_src ["stitched_y"] = tmp_points_nu_R_2 ["stitched_y"]
        tmp_src ["stitched_z"] = tmp_points_nu_R_2 ["stitched_z"]
        tmp_src ["mapped_x"] = tmp_points_nu_R_2 ["mapped_x"]
        tmp_src ["mapped_y"] = tmp_points_nu_R_2 ["mapped_y"]
        tmp_src ["mapped_z"] = tmp_points_nu_R_2 ["mapped_z"]
        
        #print(tmp_src)

        #tmp_src ["atlas_id"] = tmp_points_nu_R_2 ["atlas_id"] # 0 のままにしておく

        #print(len(tmp_src))
        del  tmp_points_intensities_, tmp_points_nu_R_,  tmp_points_nu_R_2
        gc.collect()
        
    except Exception as e:
        print("emp")
        return np.empty((0, len(dt_annotation2.names)))
    
    return tmp_src

# HDoGやnu Rをやりなおすまえにintensitiesしていることがあるので、これではダメ。
#def process_file_all (fw_dir_nu_, file_name):
#    return np.fromfile(os.path.join(fw_dir_nu_, file_name), dtype=dt_annotated_classified_intensity)

In [3]:
conditions = {
    "WT_1m": ["#4_APPmodel_Ctr1m_1_2022_1104_1550","#4_APPmodel_Ctr1m_2_2022_1108_959","#4_APPmodel_Ctr1m_3_2022_1209_1751"],
    "WT_2m": ["8w_B6JWT_atlas_1_2022_0924_1847","8w_B6JWT_atlas_2_2022_0926_1326", "8w_B6JWT_atlas_3_2022_1005_1110", "8w_B6JWT_atlas_4_2022_1010_2030","8w_B6JWT_atlas_5_2023_0423_1604",],
    "WT_3m": ["#5_APPmodel_Ctr3m_1_2023_0113_1230", "#5_APPmodel_Ctr3m_2_2023_0210_1142", "#5_APPmodel_Ctr3m_3_2023_0227_1605", "#4_APPmodel_Ctr3m_4_2023_0427_1444",],
    "WT_5m": ["#4_APPmodel_Ctr5m_1_2022_1026_252", "#4_APPmodel_Ctr5m_2_2022_1028_2336", "#4_APPmodel_Ctr5m_3_2022_1101_140", "#4_APPmodel_Ctr5m_4_2023_0512_2057"],
    "WT_7m": [ "#5_APPmodel_Ctr7m_1_2023_0206_1534","#5_APPmodel_Ctr7m_2_2023_0313_1111","#5_APPmodel_Ctr7m_3_2023_0331_2139","#4_APPmodel_Ctr7m_4_2023_0516_1559","#5_APPmodel_Ctr7m_5_2023_0522_2218"],
    "WT_9m": [ "#5_APPmodel_Ctr9m_1_2023_0130_1607", "#5_APPmodel_Ctr9m_2_2023_0216_2052" ,"#5_APPmodel_Ctr9m_3_2023_0322_2014","#5_APPmodel_Ctr9m_4_2023_0518_1449"],
    "WT_12m": ["56w_B6JWT_atlas_1_2022_1128_1548", "56w_B6JWT_atlas_2_2022_1221_1537", "56w_B6JWT_atlas_3_2022_1224_1718"],
    
    "APP_1m": ["#4_APPmodel_APP1m_1_2022_1102_1304", "#4_APPmodel_APP1m_2_2022_1106_1708","#4_APPmodel_APP1m_3_2022_1213_2151","#4_APPmodel_APP1m_4_2023_0417_2052", ],
    "APP_3m": [ "#5_APPmodel_APP3m_2_2023_0123_1645", "#5_APPmodel_APP3m_3_2023_0212_2217","#5_APPmodel_APP3m_4_2023_0214_2027","#4_APPmodel_APP3m_5_2023_0502_1423",],
   "APP_5m": ["#4_APPmodel_APP5m_1_2022_1024_1336", "#4_APPmodel_APP5m_2_2022_1027_1333","#4_APPmodel_APP5m_3_2022_1030_1257", "#4_APPmodel_APP5m_4_2023_0509_1021"],
    "APP_7m": ["#5_APPmodel_APP7m_2_2023_0315_1540","#5_APPmodel_APP7m_3_2023_0310_1401","#5_APPmodel_APP7m_4_2023_0317_1630", "#5_APPmodel_APP7m_5_2023_0520_1723"],
    "APP_9m": ["#5_APPmodel_APP9m_1_2023_0117_1352","#5_APPmodel_APP9m_2_2023_0327_1425", "#5_APPmodel_APP9m_3_2023_0329_1611", "#5_APPmodel_APP9m_4_2023_0403_1443" ],
       
    "VCP_mut_2m": ["VCPmodel_E524A_0.5x_1_2022_0624_1303", "VCPmodel_E524A_0.5x_2_2022_0626_1338", "VCPmodel_E524A_0.5x_4_2022_0922_1217", "VCPmodel_E524A_0.5x_6_2022_0628_1157"], 
   "VCP_WT_2m": ["VCPmodel_WT_0.5x_1_2022_0625_1104","VCPmodel_WT_0.5x_2_2022_0917_1808", "VCPmodel_WT_0.5x_3_2022_0627_1257", "VCPmodel_WT_0.5x_4_2022_0921_950","VCPmodel_WT_0.5x_5_2022_0923_1454","VCPmodel_WT_0.5x_6_2022_0929_953"],
    "newVCP_mut_2m": ["#5_VCPmodel_E524A_0.5x_1_2023_0120_1929","#5_VCPmodel_E524A_0.5x_2_2023_0407_1031", "#5_VCPmodel_E524A_0.5x_3_2023_0413_2036"], 
   "newVCP_WT_2m": ["#5_VCPmodel_WT_0.5x_1_2023_0405_1108","#5_VCPmodel_WT_0.5x_2_2023_0115_1852","#5_VCPmodel_WT_0.5x_3_2023_0410_1246"], 
    
    "TMT_2m": ["#3_TMTmodel_TMT3.0_8w_1_2022_1014_1410", "#3_TMTmodel_TMT3.0_8w_2_2022_1020_938", "#3_TMTmodel_TMT3.0_8w_3_2022_1023_501", "#3_TMTmodel_TMT_8w_5_2022_1205_1147",],
  "Saline_2m": ["#3_TMTmodel_Saline_8w_1_2022_1016_1318", "#3_TMTmodel_Saline_8w_2_2022_1018_2103","#3_TMTmodel_Saline_8w_3_2022_1021_1910", "#3_TMTmodel_Saline_8w_4_2022_1202_1349", "#3_TMTmodel_Saline_8w_5_2022_1217_1723", "#3_TMTmodel_Saline_8w_6_2022_1216_001"],
    
}

In [ ]:
Samples = [ "Neuronomics/" + path for paths in conditions.values() for path in paths]
Samples

In [4]:

from scipy.spatial import cKDTree
from concurrent.futures import ProcessPoolExecutor
import gc

dt_annotation2 = np.dtype([
  ('stitched_x', 'f4'), ('stitched_y', 'f4'), ('stitched_z', 'f4'),
  ('mapped_x', 'f4'), ('mapped_y', 'f4'), ('mapped_z', 'f4'),
  ('SCA_x', 'f4'), ('SCA_y', 'f4'), ('SCA_z', 'f4'),
  ('atlas_id', 'u2')
])

dt_simple2 = np.dtype([
    ('mapped_x', 'f4'), ('mapped_y', 'f4'), ('mapped_z', 'f4'), 
    ('NeuN_positive','f4'),('Iba1_positive','f4'),('atlas_id', 'u2')
])

dt_SCA = np.dtype([
  ('SCA_x', 'f4'), ('SCA_y', 'f4'), ('SCA_z', 'f4'),
  ('NeuN_positive','f4'),('Iba1_positive','f4'),('atlas_id', 'u2')
])

dt_voxel = np.dtype([
  ('X(um)', 'f4'), ('Y(um)', 'f4'), ('Z(um)', 'f4'),
  ('NeuN_neighbor_r_100um','f4'), ('atlasID', 'u2')
])

dt_registration = np.dtype([
    ('z', 'f4'), ('y', 'f4'), ('x', 'f4')
]) # 50 um scale


NeuN_cluster_threshold_list = [0.80] # だいたいが0.75のはず
NeuN_cluster_noise_threshold_list = [0.55] # だいたいが0.75のはず
Iba1_cluster_threshold_list = [0.55] # だいたいが0.45のはず

unique_id_ventricle = np.array([81, 98, 108, 129, 140, 145, 153, 164])

# img_annot から unique_id_ventricle の ID を含む要素を除外するマスクを作成
mask_without_ventricle = ~np.isin(img_annot, unique_id_ventricle)

#all_paths = ["1w_B6JWT_atlas_2_2022_1219_1829"]
import datetime

start_time = datetime.datetime.now()

print("Start time:", start_time)

for Sample in Samples:
    try:
        # stitching後 & ANTs後のcellome情報を読み出し
        print("\n")
        start_time = datetime.datetime.now()
        start_time2 = datetime.datetime.now()
        print("Start time:", start_time)

        param_path = "/home/mitani/CUBIC-informatics/param/" + Sample + "/param_multichannel-rank.json"

        print(param_path)

        with open(param_path) as f:
            param = json.load(f)

        fw_dir_nu = param["dst_basedir"]


        print("### Read parameter file ###")

        SCA_cells = np.fromfile(fw_dir_nu + "/SCA_registered_cell_coordinates_ver2.bin", dtype=dt_annotation2)

        print(len(SCA_cells))

        Classified_cells = np.fromfile(fw_dir_nu + "/For_PDF_clustering_base_data_classified_NeuN_" + str(NeuN_cluster_threshold_list[0]) +
                 "_noise_"+str(NeuN_cluster_noise_threshold_list[0]) +
                 "_Iba1_" +str(Iba1_cluster_threshold_list[0])  +"_2.bin", dtype = dt_simple2 )
        print(len(Classified_cells))

        new_array = []

        SCA_cells_classified = SCA_cells[Classified_cells['NeuN_positive']>0] 

        new_array =  np.zeros((len(SCA_cells_classified),), dtype=dt_registration)
        new_array["z"]  = SCA_cells_classified["stitched_z"] 
        new_array["y"]  = SCA_cells_classified["stitched_y"] 
        new_array["x"]  = SCA_cells_classified["stitched_x"] 


        del SCA_cells, Classified_cells, SCA_cells_classified 
        gc.collect()

        scale = 1/50 

        if Sample == "Neuronomics/8w_B6JWT_atlas_1_2022_0924_1847":
            print("8w-1 atlas")
            depth_ori = 243 
            height_ori = 440 
            width_ori = 339 
            new_array["y"]  -= 17*80
            new_array["x"]  -= 40*80

            scalemerged_img = tifffile.imread(param["dst_basedir"].replace("intensities_", "scalemerged_/img_density.tif"))
            mask_ = scalemerged_img > 0

        if Sample != "Neuronomics/8w_B6JWT_atlas_1_2022_0924_1847":
            print("Not 8w-1 atlas")

            scalemerged_img = tifffile.imread(param["dst_basedir"].replace("intensities_", "scalemerged_/img_density.tif"))

            depth_ori = scalemerged_img.shape[0]
            height_ori =scalemerged_img.shape[1]
            width_ori = scalemerged_img.shape[2]

            mask_ = scalemerged_img > 0

        print(depth_ori)
        print(height_ori)
        print(width_ori)

        img_filename_Nuclear_Isocortex_to_ori = param["dst_basedir"].replace("intensities_", "scalemerged_/img_density_NeuN.tif")

        img_N_ori,_ = np.histogramdd(
            np.vstack([
                new_array["z"] * scale,
                new_array["y"] * scale,
                new_array["x"] * scale,
                ]).T,
                bins=(depth_ori, height_ori, width_ori),
                range=[(0,depth_ori),(0,height_ori),(0,width_ori)]
            )

        img_N_ori = img_N_ori * mask_

        tifffile.imsave(
                img_filename_Nuclear_Isocortex_to_ori,
                img_N_ori.astype(np.uint16)
            )

        del  new_array, img_N_ori, mask_
        gc.collect()

        # 終了時間を記録して表示
        end_time = datetime.datetime.now()
        print("End time:", end_time)

        # 処理にかかった時間を計算して表示
        print("Duration:", end_time - start_time)

    except:
        print(str(Sample) + "is skipped.")

Start time: 2024-05-10 11:16:00.047437


Start time: 2024-05-10 11:16:00.048407
/home/mitani/CUBIC-informatics/param/Neuronomics/#4_APPmodel_Ctr5m_3_2022_1101_140/param_multichannel-rank.json
### Read parameter file ###
95525176
95525176
Not 8w-1 atlas
268
500
383
End time: 2024-05-10 11:16:18.962835
Duration: 0:00:18.914428


Start time: 2024-05-10 11:16:18.963020
/home/mitani/CUBIC-informatics/param/Neuronomics/#4_APPmodel_Ctr5m_2_2022_1028_2336/param_multichannel-rank.json
### Read parameter file ###
99955123
99955123
Not 8w-1 atlas
259
548
429
End time: 2024-05-10 11:16:38.221507
Duration: 0:00:19.258487


Start time: 2024-05-10 11:16:38.221698
/home/mitani/CUBIC-informatics/param/Neuronomics/#4_APPmodel_APP5m_1_2022_1024_1336/param_multichannel-rank.json
### Read parameter file ###
99281364
99281364
Not 8w-1 atlas
284
546
476
End time: 2024-05-10 11:16:57.863171
Duration: 0:00:19.641473
